In [2]:
import pyspark.sql.functions as sql
import pyspark.sql.types as types
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [3]:
# run nltk.download("stopwords")
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words("english"))

def tokenize(document):
    tokens = tokenizer.tokenize(document.lower())
    return [w for w in tokens if not w in stop_words]

tokenize_udf = sql.udf(tokenize, types.ArrayType(
                                                types.StringType()
                                                )
                      )

tokenize("The quickish brown fox'es jumped over the lazy-dog.")

['quickish', 'brown', 'fox', 'es', 'jumped', 'lazy', 'dog']

In [4]:
bhl_df_version = "20160516"
bhl_df = sqlContext.read.load("/guoda/data/bhl-{0}.parquet"
                              .format(bhl_df_version))

In [6]:
bhl_df.printSchema()

root
 |-- itemid: integer (nullable = true)
 |-- titleid: integer (nullable = true)
 |-- thumbnailpageid: integer (nullable = true)
 |-- barcode: string (nullable = true)
 |-- marcitemid: string (nullable = true)
 |-- callnumber: string (nullable = true)
 |-- volumeinfo: string (nullable = true)
 |-- itemurl: string (nullable = true)
 |-- localid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- institutionname: string (nullable = true)
 |-- zquery: string (nullable = true)
 |-- creationdate: date (nullable = true)
 |-- ocrtext: string (nullable = true)



In [5]:
count = bhl_df.count()

In [11]:
# crashes
#sample_bhl_df = bhl_df.sample(withReplacement=False, fraction=100.0/count)
#sample_bhl_df.count()
#sample_bhl_df.write.mode("overwrite").parquet("/guoda/data/bhl-1k-{}.parquet".format(bhl_df_version))

In [5]:
(bhl_df
 .filter(sql.substring(bhl_df["barcode"], 6, 2) == '11')
 .write
 .mode("overwrite")
 .parquet("/guoda/data/bhl-tiny-{}.parquet".format(bhl_df_version))
 )

KeyboardInterrupt: 

In [ ]:
sample_bhl_df = sqlContext.read.load("/guoda/data/bhl-tiny-{}.parquet"
                              .format(bhl_df_version))

In [ ]:
idb_df_version = "20160516"